In [1]:
import numpy as np; import pandas as pd
import xarray as xr; import matplotlib.pyplot as plt;
from assemble_class import assemble
import sqlite3

In [2]:
conn = sqlite3.connect('LoaderDB3.db') # connection
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [3]:
def Prepare_id(**kwargs):
    Data = "SELECT * FROM OBJ_ID"
    i = 0
    for key, value in kwargs.items():
        if i == 0:
            Data += " WHERE "
        else:
            Data += " AND "
        Data += "{}='{}'".format(key, value)
        i += 1
    Data += ";"
    return c.execute(Data)


def format_change(DICT):
    limits_keys = ['t0','t1','lon','lat']
    for n in range(len(DICT)):
        DICT[n]['limits'] = {key: DICT[n][key] for key in DICT[n] if key in limits_keys}
        for key in limits_keys: del DICT[n][key]

In [4]:
def calis():
    c.execute

In [5]:
Prepare_id(t0= '2018 Sep 15', name = 'SIO1')

In [6]:
sio1_id = [dict(row) for row in c.fetchall()] 

In [7]:
sio1_id

[{'name': 'SIO1',
  'type': 'MOORING',
  'project': 'PISTON',
  't0': '2018 Sep 15',
  't1': '2019 Oct 31',
  'lon': None,
  'lat': None,
  'COMBINER_ID': 102}]

In [8]:
format_change(sio1_id)

In [9]:
sio1_id[0]

{'name': 'SIO1',
 'type': 'MOORING',
 'project': 'PISTON',
 'COMBINER_ID': 102,
 'limits': {'t0': '2018 Sep 15',
  't1': '2019 Oct 31',
  'lon': None,
  'lat': None}}

In [10]:
combiner_id = sio1_id[0]['COMBINER_ID']

In [11]:
print("hola %d" %combiner_id)

hola 102


In [12]:
c.execute('SELECT * FROM COMBINER AS c INNER JOIN DATASOURCES as d ON c.Data_obj = d.Data_obj WHERE c.COMBINER_ID = %d' %combiner_id)
result  = [dict(row) for row in c.fetchall()] 

In [13]:
result

[{'COMBINER_ID': 102,
  'DATA_OBJ': 201,
  'Source': 'chipod',
  'Filepath': 'sio1_chipod.mat'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 202,
  'Source': 'chiflag',
  'Filepath': 'SIO1_chipod_finflag.csv'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 301,
  'Source': 'ADCP',
  'Filepath': 'SIO1_4021.nc'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 302,
  'Source': 'ADCP',
  'Filepath': 'SIO1_11181.nc'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 401,
  'Source': 'Temperature',
  'Filepath': 'SIO1_Tgrid.mat'}]

In [14]:
c.execute('SELECT * FROM COMBINER AS c INNER JOIN DATASOURCES as d ON c.Data_obj = d.Data_obj WHERE c.COMBINER_ID = %d AND d.Source = "chipod"' %combiner_id)
result2  = [dict(row) for row in c.fetchall()] 

In [15]:
result2

[{'COMBINER_ID': 102,
  'DATA_OBJ': 201,
  'Source': 'chipod',
  'Filepath': 'sio1_chipod.mat'}]

In [16]:
result[0]

{'COMBINER_ID': 102,
 'DATA_OBJ': 201,
 'Source': 'chipod',
 'Filepath': 'sio1_chipod.mat'}

In [17]:
for kk in range(len(result)):
    print(result[kk])

{'COMBINER_ID': 102, 'DATA_OBJ': 201, 'Source': 'chipod', 'Filepath': 'sio1_chipod.mat'}
{'COMBINER_ID': 102, 'DATA_OBJ': 202, 'Source': 'chiflag', 'Filepath': 'SIO1_chipod_finflag.csv'}
{'COMBINER_ID': 102, 'DATA_OBJ': 301, 'Source': 'ADCP', 'Filepath': 'SIO1_4021.nc'}
{'COMBINER_ID': 102, 'DATA_OBJ': 302, 'Source': 'ADCP', 'Filepath': 'SIO1_11181.nc'}
{'COMBINER_ID': 102, 'DATA_OBJ': 401, 'Source': 'Temperature', 'Filepath': 'SIO1_Tgrid.mat'}


In [18]:
sio1_paths = {}

In [19]:
sio1_paths

{}

In [20]:
sio1_data_dir = '/mnt/sda1/SciData/PISTON/Moorings/SIO1/'

In [21]:
for kk in range(len(result)):
    if result[kk]['Source'] not in sio1_paths:
        if result[kk]['Source'] != 'ADCP':
            sio1_paths[result[kk]['Source']] = [sio1_data_dir + result[kk]['Filepath']]
        else:
            sio1_paths[result[kk]['Source']] = [sio1_data_dir + 'ADCP/' + result[kk]['Filepath']]
    elif result[kk]['Source'] in sio1_paths:
        if result[kk]['Source'] != 'ADCP':
            sio1_paths[result[kk]['Source']].append(sio1_data_dir + result[kk]['Filepath'])
        else:
            sio1_paths[result[kk]['Source']].append(sio1_data_dir + 'ADCP/' + result[kk]['Filepath'])

In [22]:
sio1_paths

{'chipod': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/sio1_chipod.mat'],
 'chiflag': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_chipod_finflag.csv'],
 'ADCP': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADCP/SIO1_4021.nc',
  '/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADCP/SIO1_11181.nc'],
 'Temperature': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_Tgrid.mat']}

In [23]:
def path_maker(DataSources, data_dir):
    Dir = {}
    for kk in range(len(DataSources)):
        if DataSources[kk]['Source'] not in Dir:
            if DataSources[kk]['Source'] != 'ADCP':
                Dir[DataSources[kk]['Source']] = [data_dir + DataSources[kk]['Filepath']]
            else:
                Dir[DataSources[kk]['Source']] = [data_dir + 'ADCP/' + DataSources[kk]['Filepath']]
        elif DataSources[kk]['Source'] in Dir:
            if DataSources[kk]['Source'] != 'ADCP':
                Dir[DataSources[kk]['Source']].append(data_dir + DataSources[kk]['Filepath'])
            else:
                Dir[DataSources[kk]['Source']].append(data_dir + 'ADCP/' + DataSources[kk]['Filepath'])
    return Dir

In [24]:
sio1_paths2 = path_maker(result,sio1_data_dir)

In [25]:
sio1_paths2

{'chipod': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/sio1_chipod.mat'],
 'chiflag': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_chipod_finflag.csv'],
 'ADCP': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADCP/SIO1_4021.nc',
  '/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADCP/SIO1_11181.nc'],
 'Temperature': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_Tgrid.mat']}

In [26]:
def id_maker(**kwargs):
    Prepare_id(**kwargs)
    A = [dict(row) for row in c.fetchall()]
    format_change(A)
    return A

In [27]:
def Data_extraction(obj_id, n, **kwargs):
    combiner_id = obj_id[n]['COMBINER_ID']
    common_str = 'SELECT * FROM COMBINER AS c INNER JOIN DATASOURCES as d ON c.Data_obj = d.Data_obj WHERE c.COMBINER_ID = %d'
    i = 1
    for key, value in kwargs.items():
        common_str += " AND d."
        common_str += "{}='{}'".format(key, value)
        i += 1
    common_str += ";"
    c.execute(common_str %combiner_id)
    return [dict(row) for row in c.fetchall()] 

In [28]:
Data_extraction(sio1_id, 0, Source='ADCP')

[{'COMBINER_ID': 102,
  'DATA_OBJ': 301,
  'Source': 'ADCP',
  'Filepath': 'SIO1_4021.nc'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 302,
  'Source': 'ADCP',
  'Filepath': 'SIO1_11181.nc'}]